# LA POULE QUI CHANTE - Pré-traitement des données

# Introduction

In [1]:
## Importation des modules & librairies

import numpy as np # Calculs numériques et opérations mathématiques
import pandas as pd # Manipulation et analyse de données

import matplotlib.pyplot as plt # Création de graphiques et visualisations
import plotly.express as px # Visualisation interactive des données
import seaborn as sns # Visualisations statistiques avancées

## Paramètres d'affichages
pd.set_option('display.float_format', lambda x: '%.0f' % x) # Affichage numérique

In [2]:
## Mise en place des fonctions
def explore_dataframe(data_to_analyse):
    """
    Effectue une analyse exploratoire basique sur un dataframe donné.

    Parameters:
    data_to_analyse (DataFrame): Le dataframe à analyser.
    """
    # Dimensions du dataframe
    print('*' * 40 + " DIMENSIONS " + '*' * 44)
    print(f"Le dataframe comporte {data_to_analyse.shape[1]} variable(s) pour {data_to_analyse.shape[0]} observation(s).")

    # Typologies des variables
    print('*' * 40 + " TYPOLOGIES " + '*' * 44)
    for col, dtype in data_to_analyse.dtypes.items():
        print(f"{col} : {dtype}")

    # Effectifs des variables
    print('*' * 40 + " EFFECTIFS " + '*' * 45)
    for col, count in data_to_analyse.describe(include='all').loc['count'].items():
        print(f"{col} : {count}")

    # Valeurs nulles dans le dataframe
    print('*' * 40 + " VALEURS NULLES " + '*' * 40)
    for col, count in data_to_analyse.isnull().sum().items():
        print(f"{col} : {count}")

    # Duplicats dans le dataframe
    print('*' * 40 + " DUPLICATS " + '*' * 45)
    for col, count in data_to_analyse[data_to_analyse.duplicated()].count().items():
        print(f"{col} : {count}")


In [3]:
## Importation des données initiales

# Importation des fichiers FAO
population_data = pd.read_csv('Population_2000_2018.csv')
disponibilite_data = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
# Importation des fichiers issues de l'analyse PESTEL
stabilite_data = pd.read_csv('pib_stabilite_politique_2017.csv')

# Pré-traitement des données

## Données géo-politiques

In [4]:
## Analyse exploratoire
explore_dataframe(stabilite_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 7 variable(s) pour 384 observation(s).
**************************************** TYPOLOGIES ********************************************
Domaine : object
Zone : object
Élément : object
Produit : object
Année : int64
Unité : object
Valeur : float64
**************************************** EFFECTIFS *********************************************
Domaine : 384
Zone : 384
Élément : 384
Produit : 384
Année : 384.0
Unité : 384
Valeur : 384.0
**************************************** VALEURS NULLES ****************************************
Domaine : 0
Zone : 0
Élément : 0
Produit : 0
Année : 0
Unité : 0
Valeur : 0
**************************************** DUPLICATS *********************************************
Domaine : 0
Zone : 0
Élément : 0
Produit : 0
Année : 0
Unité : 0
Valeur : 0


In [5]:
## Préparation des données

# Selection des variables pertinentes
stabilite_data = stabilite_data[['Zone', 'Produit', 'Valeur']]

# Faire pivoter le dataframe
stabilite_data = pd.pivot(stabilite_data,index=['Zone'],columns=['Produit'],values=['Valeur'])

# Suppression d'un niveau d'index pour simplifier la structure des données
stabilite_data.columns = stabilite_data.columns.droplevel()

# Modifier le nom des variables
stabilite_data = stabilite_data.rename(columns={'PIB par habitant, ($ PPA internationaux constants de 2017)': 'PIB par habitant',
             'Stabilité politique et absence de violence/terrorisme (indice)': 'Stabilité politique'})

# Aperçu des valeurs nulles
# stabilite_data[stabilite_data.isna().any(axis=1)]
# print("On constate la présence de valeurs manquantes, elles seront traitées par la suite.")

# Analyse descriptive
stabilite_data.describe()

Produit,PIB par habitant,Stabilité politique
count,187,197
mean,20925,-0
std,22086,1
min,774,-3
25%,4863,-1
50%,13102,0
75%,29087,1
max,126144,2


In [6]:
## Top 5 PIB par habitant
stabilite_data.sort_values(by='PIB par habitant', ascending=False).head(5)

Produit,PIB par habitant,Stabilité politique
Zone,,
Chine - RAS de Macao,126144,1
Luxembourg,114986,1
Singapour,95310,2
Qatar,91739,1
Bermudes,81835,1


In [7]:
## Flop 5 PIB par habitant
stabilite_data.sort_values(by='PIB par habitant', ascending=True).head(5)

Produit,PIB par habitant,Stabilité politique
Zone,,
Burundi,774,-2
République centrafricaine,913,-2
République démocratique du Congo,1060,-2
Somalie,1079,-2
Niger,1163,-1


## Données démographiques

In [8]:
## Analyse exploratoire
explore_dataframe(population_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 15 variable(s) pour 4411 observation(s).
**************************************** TYPOLOGIES ********************************************
Code Domaine : object
Domaine : object
Code zone : int64
Zone : object
Code Élément : int64
Élément : object
Code Produit : int64
Produit : object
Code année : int64
Année : int64
Unité : object
Valeur : float64
Symbole : object
Description du Symbole : object
Note : object
**************************************** EFFECTIFS *********************************************
Code Domaine : 4411
Domaine : 4411
Code zone : 4411.0
Zone : 4411
Code Élément : 4411.0
Élément : 4411
Code Produit : 4411.0
Produit : 4411
Code année : 4411.0
Année : 4411.0
Unité : 4411
Valeur : 4411.0
Symbole : 4411
Description du Symbole : 4411
Note : 258
**************************************** VALEURS NULLES ****************************************
Code Domaine :

In [9]:
## Préparation des données

# Selection des données pertinentes
population_data = population_data.loc[population_data['Année'] == 2017, :]

# Selection des variables pertinentes
population_data = population_data[['Zone', 'Valeur']]

# Modifier le nom des variables
population_data = population_data.rename(columns={'Valeur' : 'Population'})

# Mise à l'échelle de la variable "Population"
population_data['Population'] = population_data['Population'] * 1000

# Modifier la typologie des variables
population_data['Population'] = population_data['Population'].astype('int64')

In [10]:
# Analyse descriptive
population_data.describe()

,Population
count,236
mean,31983619
std,131894885
min,793
25%,380303
50%,5203510
75%,19308418
max,1421021791


In [11]:
## Top 5
population_data.sort_values(by='Population', ascending=False).head()

,Zone,Population
823,"Chine, continentale",1421021791
1952,Inde,1338676785
1230,États-Unis d'Amérique,325084756
1971,Indonésie,264650963
3029,Pakistan,207906209


In [12]:
## Recherche d'outliers

# Flop 50
# population_data.sort_values(by='Population', ascending=True).head(50)

# Détection d'outliers potentiels

# Préparation des données pour le graphique
fig = px.box(population_data, y='Population', title='Boxplot des Populations')

# Affichage du graphique
fig.show()

Les cas de la Chine continentale et de l'Inde seront considérés comme outlier au sein de cette analyse.

De plus, il a été décidé de ne pas répertorier les pays repertoriant moins de 250 000 habitants.



In [13]:
## Suppression des pays dont la population est inférieur à 250 000 habitants
population_data = population_data.loc[population_data['Population'] > 250000, :]

## Données relatives aux disponibilités alimentaires

In [14]:
## Analyse exploratoire
explore_dataframe(disponibilite_data)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 14 variable(s) pour 176600 observation(s).
**************************************** TYPOLOGIES ********************************************
Code Domaine : object
Domaine : object
Code zone : int64
Zone : object
Code Élément : int64
Élément : object
Code Produit : int64
Produit : object
Code année : int64
Année : int64
Unité : object
Valeur : float64
Symbole : object
Description du Symbole : object
**************************************** EFFECTIFS *********************************************
Code Domaine : 176600
Domaine : 176600
Code zone : 176600.0
Zone : 176600
Code Élément : 176600.0
Élément : 176600
Code Produit : 176600.0
Produit : 176600
Code année : 176600.0
Année : 176600.0
Unité : 176600
Valeur : 176600.0
Symbole : 176600
Description du Symbole : 176600
**************************************** VALEURS NULLES ****************************************
Code Doma

In [15]:
## Préparation des données

# Selection des variables pertinentes
disponibilite_data = disponibilite_data[['Zone', 'Élément', 'Produit', 'Valeur']]

# Aperçu des valeurs possibles au sein de la variable "Élément"
# disponibilite_data['Élément'].unique()

# Selection des données pertinentes
liste_elements = (
    'Production',
    'Importations - Quantité',
    'Variation de stock',
    'Exportations - Quantité',
    'Disponibilité intérieure',
    'Nourriture',
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité de protéines en quantité (g/personne/jour)')
disponibilite_data = disponibilite_data.loc[disponibilite_data['Élément'].isin(liste_elements), :]

# Aperçu des valeurs possibles au sein de la variable "Produit"
# disponibilite_data['Produit'].unique()
# print("Pour cette étude de marché, seul la " Viande de volailles " nous intéresse.")

# Selection des données pertinentes
disponibilite_data = disponibilite_data.loc[disponibilite_data['Produit'] == 'Viande de Volailles', :]

# Faire pivoter le dataframe
disponibilite_data = pd.pivot(disponibilite_data, index=['Zone'], columns=['Élément'], values=['Valeur'])

# Suppression d'un niveau d'index pour simplifier la structure des données
disponibilite_data.columns = disponibilite_data.columns.droplevel()

Il y a 37 valeurs nulles compatibilisées au sein de la variable "Exportations - Quantité".

Deux choix sont possibles :

- Remplacer les valeurs nulles par 0
- Appliquer une formule de calcul à l'aide d'informations déjà présentes

Source : https://www.fao.org/3/X9892F/x9892f03.html

In [16]:
## Gestion des valeurs nulles

# Identification des valeurs nulles potentielles
# disponibilite_data.isna().sum()

# Calcul 1 : (Production + Importations - Disponibilité alimentaire en quantité + Variation de Stock)
# Calcul 2 : Résultat du calcul 1 - Disponibilité intérieure

disponibilite_data['Exportations - Quantité'] = disponibilite_data.apply(
    lambda x : ((x['Production'] + x['Importations - Quantité'] -
                 x['Disponibilité alimentaire en quantité (kg/personne/an)'] +
                 x['Variation de stock'])
                - x['Disponibilité intérieure'])
    if pd.isnull(x['Exportations - Quantité']) else x['Exportations - Quantité'], axis=1)

# Identification des valeurs nulles potentielles restantes
disponibilite_data.isna().sum()

# print("La formule a pu remplacer une grande partie des valeurs manquantes.")
# print("Il reste cependant des valeurs nulles au sein de la variable exportation, potentiellement dues aux informations manquantes au sein d'autres variables.")

# Aperçu des valeurs nulles
print("Valeurs nulles : Ces dernières seront traitées par la suite.")
print("")
disponibilite_data[disponibilite_data.isna().any(axis=1)]



Valeurs nulles : Ces dernières seront traitées par la suite.



Élément,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock
Zone,,,,,,,,,
Djibouti,9,3,1,3,NaN,3,3,NaN,0
Maldives,47,14,5,12,NaN,12,7,NaN,0
Ouzbékistan,6,2,1,NaN,NaN,NaN,NaN,NaN,NaN
Pérou,62,13,7,1523,1,60,424,1465,NaN
République démocratique populaire lao,41,11,4,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Analyse descriptive
disponibilite_data.describe()

Élément,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock
count,172,172,172,170,168,170,170,168,169
mean,75,20,7,688,105,90,657,725,14
std,61,16,6,2187,464,187,2137,2501,75
min,0,0,0,2,-70,0,2,0,-119
25%,22,6,2,30,0,3,28,14,0
50%,64,18,7,100,1,16,100,70,0
75%,106,30,10,368,17,81,365,410,7
max,243,72,28,18266,4223,1069,18100,21914,859


In [18]:
## Top 5 en matière d'exportations
disponibilite_data.sort_values(by='Exportations - Quantité', ascending=False).head()

Élément,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Production,Variation de stock
Zone,,,,,,,,,
Brésil,204,48,16,9982,4223,3,9982,14201,0
États-Unis d'Amérique,219,56,20,18266,3692,123,18100,21914,80
Pays-Bas,70,20,8,372,1418,608,346,1100,-82
Pologne,107,30,12,1156,1025,55,1150,2351,225
Thaïlande,47,13,4,881,796,2,896,1676,1


In [19]:
## Recherche d'outliers
# Détection d'outliers potentiels

# Préparation des données pour le graphique
fig = px.box(disponibilite_data, y='Exportations - Quantité', title='Boxplot des Exportations')

# Affichage du graphique
fig.show()

Le Brésil et les Etats-Unis seront considérés à part, du à leurs valeurs extrêmes à différente(s) variable(s), notamment les Exportations.

Il y a donc actuellement 4 pays qui nécessitent un traitement spécial pour cette etude de marché :

- Chine, continentale
- Inde
- Brésil
- États-Unis

# Fusion et exportation des données

In [35]:
## Fusion des données
# Fusion externe ( Outer ) pour conserver l'intégralité des données.

# Fusion des données géo-politiques et des données relatives aux disponibilités alimentaires
df_dispo_alimentaire_stabilite_politique = pd.merge(disponibilite_data, stabilite_data, on='Zone', how='outer')

# Ajout des données démographiques
df = pd.merge(df_dispo_alimentaire_stabilite_politique, population_data, on='Zone', how='outer')

# Selection des variables pertinentes
df = df[['Zone',
        'Disponibilité alimentaire en quantité (kg/personne/an)',
        'Disponibilité de protéines en quantité (g/personne/jour)',
        'Disponibilité intérieure',
        'Exportations - Quantité',
        'Importations - Quantité',
        'PIB par habitant',
        'Stabilité politique',
        'Population']]

In [36]:
## Analyse exploratoire
explore_dataframe(df)

**************************************** DIMENSIONS ********************************************
Le dataframe comporte 9 variable(s) pour 208 observation(s).
**************************************** TYPOLOGIES ********************************************
Zone : object
Disponibilité alimentaire en quantité (kg/personne/an) : float64
Disponibilité de protéines en quantité (g/personne/jour) : float64
Disponibilité intérieure : float64
Exportations - Quantité : float64
Importations - Quantité : float64
PIB par habitant : float64
Stabilité politique : float64
Population : float64
**************************************** EFFECTIFS *********************************************
Zone : 208
Disponibilité alimentaire en quantité (kg/personne/an) : 172.0
Disponibilité de protéines en quantité (g/personne/jour) : 172.0
Disponibilité intérieure : 170.0
Exportations - Quantité : 168.0
Importations - Quantité : 170.0
PIB par habitant : 187.0
Stabilité politique : 197.0
Population : 187.0
*************

In [37]:
## Gestion des valeurs nulles

# Création d'un second dataframe contenant uniquement les pays dont certaines observations sont manquantes
Incomplete_Data = df[df.isnull().any(axis=1)]

# Exportation du fichier au format .csv
Incomplete_Data.to_csv('Info_incomplete_pays.csv')

# Suppression des valeurs nulles au sein du dataframe final
df.dropna(inplace=True)

# Ré-indexation des lignes
df = df.reset_index(drop=True)

In [38]:
# Affichage du nombre de pays ( Doit être supérieur à 100 )
print(f"Le nombre de pays comptabilisé est de : {df['Zone'].count()}")

# Affichage de la population comptabilisée ( Doit être supérieur à 60 % )
Population2017 = 7600000000
Pop_comptabilise = df["Population"].sum()
Pourcentage_comptabilise = Pop_comptabilise / Population2017 * 100
print(f"La population comptabilisée représente : {round(Pourcentage_comptabilise)} %")

# Exportation du fichier au format .csv
df.to_csv('Clean_Data.csv')

Le nombre de pays comptabilisé est de : 151
La population comptabilisée représente : 76 %
